In [0]:
%sql
-- ============================================================
-- CAMADA: GOLD (BI / ANALÍTICA)
-- TABELA: fct_cliente_resumo
--
-- OBJETIVO:
-- Tabela analítica por cliente, base para churn,
-- clusterização e análises comportamentais.
--
-- FONTE:
-- workspace.olist_gold.fct_pedidos
--
-- DESTINO:
-- workspace.olist_gold.fct_cliente_resumo
-- ============================================================
CREATE SCHEMA IF NOT EXISTS workspace.olist_gold;

CREATE OR REPLACE VIEW workspace.olist_gold.fct_cliente_resumo AS

WITH data_referencia AS (
    SELECT
        CAST(MAX(order_purchase_timestamp) AS DATE) AS ref_date
    FROM workspace.olist_gold.fct_pedidos
)

SELECT
    f.customer_id,

    COUNT(f.order_id) AS total_orders,

    SUM(f.total_payment_value) AS total_spent,
    AVG(f.total_payment_value) AS avg_ticket,

    MIN(f.order_purchase_timestamp) AS first_purchase_date,
    MAX(f.order_purchase_timestamp) AS last_purchase_date,
    DATEDIFF(
        MAX(d.ref_date),
        CAST(MAX(f.order_purchase_timestamp) AS DATE)
    ) AS days_since_last_purchase,

    AVG(f.review_score) AS avg_review_score

FROM workspace.olist_gold.fct_pedidos f
CROSS JOIN data_referencia d
GROUP BY f.customer_id;